In [1]:
pip install pillow tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk
from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf

# Định nghĩa các mô hình và kích thước đầu vào tương ứng
models = {
    'CNN': {'path': './sequential_model.h5', 'input_size': (224, 224, 1)},
    'VGG16': {'path': './vgg16_1.h5', 'input_size': (224, 224, 3)},
    'ResNet50': {'path': './resnet50_custom.h5', 'input_size': (224, 224, 3)},
    'DenseNet121': {'path': './DenseNet3.h5', 'input_size': (224, 224, 3)},
    'Vision Transformer': {'path': './VisionTransformer.h5', 'input_size': (224, 224, 3)}
}

# Hàm tải mô hình và sửa tên các lớp nếu cần
def load_model(model_name):
    model = tf.keras.models.load_model(models[model_name]['path'])

    # Sửa tên các lớp nếu cần
    for layer in model.layers:
        if '/' in layer.name:
            layer._name = layer.name.replace('/', '_')

    return model

# Hàm xử lý hình ảnh
def preprocess_image(image_path, input_size):
    if input_size[2] == 1:
        # Xử lý ảnh trắng đen cho mô hình CNN
        image = Image.open(image_path).convert('L')  # Chuyển đổi hình ảnh sang grayscale
        image = image.resize(input_size[:2])  # Chỉ lấy (height, width)
        image = np.array(image)
        image = np.expand_dims(image, axis=-1)  # Thêm một trục cho kênh
    else:
        # Xử lý ảnh RGB cho các mô hình khác
        image = Image.open(image_path).convert('RGB')
        image = image.resize(input_size[:2])  # Chỉ lấy (height, width)
        image = np.array(image)

    image = image / 255.0
    image = np.expand_dims(image, axis=0)
    return image

# Hàm dự đoán
def predict():
    global image_path
    if not image_path:
        result_var.set("No image selected!")
        return

    model_name = model_var.get()
    model = load_model(model_name)
    input_size = models[model_name]['input_size']
    image = preprocess_image(image_path, input_size)
    prediction = model.predict(image)
    result = 'Disease' if prediction[0][0] > 0.5 else 'Normal'
    result_var.set(result)

# Hàm chọn hình ảnh
def select_image():
    global image_path
    image_path = filedialog.askopenfilename()
    if not image_path:
        return

    image = Image.open(image_path)
    image = image.resize((300, 300))
    image = ImageTk.PhotoImage(image)
    panel.config(image=image)
    panel.image = image

# Tạo giao diện
root = tk.Tk()
root.title("Disease Detection")

panel = tk.Label(root)
panel.pack()

btn_select = tk.Button(root, text="Insert Picture", command=select_image)
btn_select.pack()

model_var = tk.StringVar()
model_var.set("CNN")

model_menu = ttk.Combobox(root, textvariable=model_var)
model_menu['values'] = list(models.keys())
model_menu.pack()

btn_predict = tk.Button(root, text="Predict", command=predict)
btn_predict.pack()

result_var = tk.StringVar()
result_label = tk.Label(root, textvariable=result_var)
result_label.pack()

image_path = None

root.mainloop()
